In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set Hyperparameters 
input_size = 28 
sequenc_length = 28 
num_layers = 2 
hidden_size = 256 
num_classes = 10 
learning_rate = 0.001 
batch_size = 64 
mun_epochs = 2 

# Create RNN 
class RNN(nn.Module): 
    def __init__(self, input_size, hidden_size, num_layers, num_classes): 
        super(RNN, self).__init__() 
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.rnn = nn.RNN(inpu_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x): 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward prop 
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out 
        
        
        
        

In [3]:
# import 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as f 
from torch.utils.data import DataLoader 
import torchvision.datasets as datasets 
import torchvision.transforms as transforms 

# Set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters 
input_size = 28 
sequence_length = 28 
num_layers = 2 
hidden_size = 256 
num_classes = 10 
learning_rate = 0.001 
batch_size = 64 
num_epochs = 2 


# Load data 
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Create RNN 
class RNN(nn.Module): 
    def __init__(self, input_size, hidden_size, num_layers, num_classes): 
        super(RNN, self).__init__() 
        self.hidden_size = hidden_size 
        self.num_layers = num_layers 
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x): 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(srlf.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward prop 
        out, _ = self.rnn(x, h0)
#         out, _ = self.gru(x, h0)
        out, _ = self.lstm(x, (h0,c0))
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        
#         out_lstm = self.fc(out[:, -1, :])   # out[all mini batch & all training sample, last hidden, all feature]
        
        return out 

# Initialize Network
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Lass and Optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the Network 
for epoch in range(num_epochs): 
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)
        
        # forward 
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient desent or adam step
        optimizer.step()

        
        
# Check accuracy 
def check_accuracy(loader, model): 
    if loader.dataset.train: 
        print("Checking accuracy on training data...")
    else:
        print("Checking accuracy on test data...")
        
    num_correct = 0 
    num_samples = 0 
    model.eval() 
    
    with torch.no_grad(): 
        for x, y in loader: 
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            
            scores = model(x)  # RNN()
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
        print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')
        
    model.train()


In [4]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data...
got 58121 / 60000 with accuracy 96.86833333333334
Checking accuracy on test data...
got 9672 / 10000 with accuracy 96.72


In [7]:
!nvidia-smi

Fri Jul  1 06:40:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   54C    P0   237W / 250W |  16038MiB / 32510MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  